# Functions

## Topics
- How to declare a function
- Duck-typing in Julia
- Mutating vs. non-mutating functions
- Some higher-order functions

## Functions in a functional language
Functions are the building blocks of Julia code, acting as the subroutines, procedures, blocks, and similar structural concepts found in other programming languages.

- A function's job is to take a tuple of values as an argument list and return a value. 
- If the arguments contain mutable values like arrays, the array can be modified inside the function. 
    - By convention, an exclamation mark (!) at the end of a function's name indicates that the function may modify its arguments.

## How to declare a function

Julia gives us a few different ways to write a function. The first (and most standard) requires `function` and `end` keywords.

In [ ]:
function sayhi(name)
    println("Hi $name, nice to meet you!")
end

sayhi("R2D2")

In [ ]:
function f(x)
    return x^2
end

f(42)

## Single line function definitions
Alternatively, we could have spared a few lines of code and written:

In [ ]:
sayhi2(name) = println("Hi $name, nice to meet you!")

In [ ]:
f2(x) = x^2

## Anonymous functions
Or we could have declared them as "anonymous" functions as

In [ ]:
sayhi3 = name -> println("Hi $name, nice to meet you!")

In [ ]:
f3 = x -> x^2

## Duck-typing in Julia
*"If it quacks like a duck, it's a duck."*

Julia functions will just work on whatever input makes sense. For example, `sayhi` works also with the name written as an integer:

In [2]:
sayhi(55595472)

Hi 55595472, nice to meet you!


And `f` will work on a matrix

In [ ]:
A = rand(3,3)
A

In [ ]:
f(A)

## Mutating vs. non-mutating functions
By convention, functions followed by `!` alter their contents and functions lacking `!` do not.

For example, let's look at the difference between `sort` and `sort!`.

In [ ]:
v = [3,5,2]

In [4]:
sort(v)

3-element Array{Int64,1}:
 2
 3
 5

In [5]:
v

3-element Array{Int64,1}:
 3
 5
 2

`sort(v)` returns a sorted array that contains the same elements as `v`, but `v` is left unchanged.

On the other hand, when we run `sort!(v)` the content of `v` is really modified.

In [ ]:
sort!(v)

In [9]:
v

3-element Array{Int64,1}:
 2
 3
 5

## Some higher-order functions: `map`

`map` is a "higher-order" function in Julia that takes a *function* as one of its input arguments. `map` then applies that function to every element of the data structure you pass. 

For example
```julia
map(f, [1,2,3])
```
will correspond to
```julia
[f(1), f(2), f(3)]
```

## Some higher-order functions: `broadcast`
`broadcast` is another higher-order function like `map`. `broadcast` is actually a generalization, so it can do what `map` but also more!

Syntax is the same
```julia
broadcast(f, [1,2,3])
```

And so we have again applied f (squared) to all elements of `[1,2,3]`. 

## Broadcasting (or vectorizing)
Some syntactic sugar for calling `broadcast` is to place `.` between the name of the function you want to broadcast and its input arguments. 
        
For example
```julia
broadcast(f, [1,2,3])
```
is the same as
```julia
f.([1,2,3])
```

Note that this is not the same as `f([1,2,3])` because we can not square a vector!

Let's try broadcasting for a matrix `A`

In [10]:
A = [i + 3j for j in 0:2, i in 1:3]

3×3 Array{Int64,2}:
 1  2  3
 4  5  6
 7  8  9

In [11]:
f(A)

3×3 Array{Int64,2}:
  30   36   42
  66   81   96
 102  126  150

In [12]:
B = f.(A)

3×3 Array{Int64,2}:
  1   4   9
 16  25  36
 49  64  81

## Dot syntax for vectorization
The dot syntax allows to write complex compound **elementwise** expressions in a way that looks natural/closer to mathematical notation. 

For example:

In [ ]:
A + 2 .* f.(A) ./ A

Instead of the more nasty looking version with `broadcast` as

In [ ]:
broadcast(x-> x + 2 * f(x) / x, A) 

### Advanced: SIMD vectorization
Vectorization is discussed more in the bonus notebook about SIMD vectorization.

In short, the topic is quite technical but you should rest assured that the dot syntax actually works quite well to make your code easy to read **and** fast to run.

## Macros
Finally, let's touch the metaprogramming capabilities of Julia. 

Since metaprogramming is a whole another topic (see bonus notebook) we will only cover the very basics of something you might encounter when dealing with Julia code: macros.

Macros provide a method to include generated code in the final body of a program. A macro maps a tuple of arguments to a returned **expression**, and the resulting expression is compiled directly.

This means that macros can change how functions work, hence the *meta* in metaprogramming.

In [7]:
macro sayhello()
    return :( println("Hello, world!") )
end

@sayhello (macro with 1 method)

## Macro invocation
Macros are invoked with the following general syntax:
```julia
@name expr1 expr2 ...
@name(expr1, expr2, ...)
```

In [8]:
@sayhello

Hello, world!


## Hold up: why macros?
So, why do macros exist?

Macros are necessary because they execute when code is parsed, therefore, macros allow the programmer to generate and include fragments of customized code before the full program is run. To illustrate the usefullness, consider the following example:

In [6]:
@time A = rand(10^3, 10^3)

  0.077456 seconds (129.70 k allocations: 14.319 MiB, 7.03% gc time)


1000×1000 Array{Float64,2}:
 0.133788  0.289875   0.301358  …  0.722712   0.132119     0.0711878 
 0.232199  0.708361   0.757317     0.382954   0.90457      0.541329  
 0.841865  0.25019    0.863874     0.791287   0.838446     0.688754  
 0.905365  0.699254   0.791713     0.168175   0.700649     0.00768163
 0.908601  0.231062   0.805617     0.274653   0.237761     0.0545861 
 0.97303   0.350525   0.859059  …  0.122305   0.480321     0.683983  
 0.485601  0.897776   0.79468      0.610637   0.542205     0.0194064 
 0.586251  0.308654   0.42805      0.674692   0.713126     0.920915  
 0.858069  0.38761    0.466336     0.115616   0.452334     0.304697  
 0.587535  0.220633   0.023312     0.282934   0.843557     0.4574    
 0.150961  0.169397   0.608804  …  0.343032   0.097016     0.852851  
 0.320002  0.302165   0.775676     0.0170152  0.495148     0.293688  
 0.141472  0.400419   0.492219     0.62024    0.493211     0.893538  
 ⋮                              ⋱                             